# Importing Libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import requests
import re
import pandas as pd
from datetime import datetime



In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import requests
import re
from datetime import datetime
import sys
import bs4

libraries = [
    pd,
    soup,
    requests,
    re,
    uReq,
    datetime
]

def get_version(lib):
    if hasattr(lib, '__version__'):
        return lib.__version__
    elif lib.__name__ == 'BeautifulSoup':
        return f"bs4 {bs4.__version__}"
    elif lib.__name__ in ['re', 'urlopen', 'datetime']:
        return f"Part of Python {sys.version.split()[0]}"
    else:
        return "Version not available"

for lib in libraries:
    print(f"{lib.__name__}: {get_version(lib)}")

print(f"\nPython version: {sys.version}")

# Building the URL

I'm using scraperapi here as a kind of VPN. Basically I am passing a url through there service so I can avoid being locked out. The google scholar url works like this:

```
https://scholar.google.ch/scholar?hl=en&q=["SPECIES NAME"]
```

In [2]:
url = "http://api.scraperapi.com?api_key=YOUR_API_KEY_HERE&url=https://scholar.google.ch/scholar?hl=en&q="

# Scraping Google Scholar

Here I am passing the url I built above and downloading the html. The psoup variable is converting it to a beautiful soup object so we can interact with it. 

In [47]:
uClient = uReq(url+name)
page_html = uClient.read()
uClient.close()
psoup = soup(page_html, "lxml")

Searching the psoup object to find the specifgic div class identified in google scholar on Chrome. It returns several results, so I use the "[1]" at the end to pick the right one in the list. Adding the ".text" removes the html tags and only returns the text. As you can see it spits out the line at the top of the search results

In [ ]:
resultsRaw = psoup.find_all('div',class_='gs_ab_mdw')[1].text
resultsRaw

Using regex to pull out just the number from the regular string. The regex expression breaks up the sentence in to three parts, first "About ", then the results (which I grab by using a period which represents any character, and an astericks, which allows for any number of characters), then " results". Selecting the third object in the list returns the number

In [49]:
count = re.search('(About )(.*)( results)',resultsRaw)[2]

In [ ]:
count

# Importing Rare Plants

In [ ]:
cnps = pd.read_csv('CNPSresult20210227.csv',encoding='latin-1')
cnps

For the moment, I am going to do a really simple analysis and just feed the names in. For future work, however, I will need to consider:
1. How does the inclusion of var. and ssp. affect results
2. How does variations of subspecies affect results

In [ ]:
names = cnps['Scientific Name']
names

# Running the list

This is a barebones script, pulling the raw text output of the results from Google Scholar. I tried cleaning it within the script but kept running into various edge cases. My current plan now is to pull what I can, record what doesn't work, and do the regex in pandas after the fact. 

In [ ]:
results = []

for spp in names:
    name = spp.replace(" ","%20")
    name = '"'+name+'"'
    
    uClient = uReq(url+name)
    page_html = uClient.read()
    uClient.close()
    psoup = soup(page_html, "lxml")
    
    try: 
        result = psoup.find_all('div',class_='gs_ab_mdw')[1].text
    except: 
        result = 'Failed'
        
    results.append([spp,result])
    print(spp,result)

The code above quit after about 500 names. So I took where it left off and wrote a new script that is a bit more robust. It will attempt to pull just the text, and if it doesnt work for whatever reason, it will move on with a simple "Failed" result.

In [ ]:
names[504:]

In [ ]:
results2 = []

for spp in names[504:]:
    name = spp.replace(" ","%20")
    name = '"'+name+'"'
    try: 
        uClient = uReq(url+name)
        page_html = uClient.read()
        uClient.close()
        psoup = soup(page_html, "lxml")

        result = psoup.find_all('div',class_='gs_ab_mdw')[1].text
    except: 
        result = 'Failed'
        
    results2.append([spp,result])
    now = datetime.now().strftime("%H:%M:%S")
    print(spp,result)

It took awhile, but it got through the list. Below I am combining the results of both scrapes. I first make a dataframe for each results list, then use pd.concat to combine into the new dataframe 'r' (for rare plants).

In [157]:
df = pd.DataFrame(results,columns=['name','result'])
df2 = pd.DataFrame(results2,columns=['name','result'])

In [ ]:
r = pd.concat([df,df2])
r

I make a quick csv to because I'm nervous

In [159]:
r.to_csv('rawResults20210228.csv')

### Scratch Regex code

In [ ]:
try:
    count = re.search('(About )(.*)( results)',resultsRaw)[2]
except:
    count = re.search('(.*)( results)',resultsRaw)[1]

Updating the r dataframe based on the add dataframe. You need to set the index on name first for it to work. 

In [165]:
r.set_index('name',inplace=True)
add.set_index('name',inplace=True)

In [168]:
r.update(add,join='left',overwrite=True)

New count of error species after the update. Matches with the add dataframe!

# New Thread
Picking this up on March 7, 2021. Working to make the script more reliable. Will mostly start fresh from this point on. 

In [24]:
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import requests
import re
import pandas as pd
from datetime import datetime

In [ ]:
cnps = pd.read_csv('CNPSresult20210227.csv',encoding='latin-1')
cnps

In [ ]:
cnps['searchName'] = cnps['Scientific Name']
cnps['searchName'] = cnps['searchName'].str.replace('var\.','*')
cnps['searchName'] = cnps['searchName'].str.replace('ssp\.','*')
cnps

In [ ]:
cnps[cnps['Scientific Name'].str.contains('ssp\.')]

In [ ]:
names = cnps['searchName']
len(names)

In [ ]:
results = []

total = len(names)
current = 0

url = "http://api.scraperapi.com?api_key=YOUR_API_KEY_HERE&url=https://scholar.google.ch/scholar?hl=en&q="

print('Starting script: '+ datetime.now().strftime("%H:%M:%S"))

for spp in names:
    name = spp.replace(" ","%20")
    name = '"'+name+'"'
    
    check = False
    for attempt in range(10):
        while check is False:
            try: 
                uClient = uReq(url+name)
                page_html = uClient.read()
                uClient.close()
                psoup = soup(page_html, "lxml")

                result = psoup.find_all('div',class_='gs_ab_mdw')[1].text
                check = len(result) > 1
            except Exception: 
                continue
            break
        
    results.append([spp,result])
    now = datetime.now().strftime("%H:%M:%S")
    current += 1
    
    print(current,'/',total,now,spp,result)

In [ ]:
df = pd.DataFrame(results, columns=['name','result'])
df

I checked each one of these results manually, and none had results. Will convert to zeros.

In [ ]:
df[~df['result'].str.contains('result')]

In [209]:
df['result'] = df['result'].replace('','0')

In [ ]:
df[~df['result'].str.contains('result')]

In [211]:
df.to_csv('scholarResults20210307.csv')

In [30]:
import pandas as pd
df = pd.read_csv('scholarResults20210307.csv')

In [ ]:
df

In [32]:
df['count'] = df['result'].str.extract('(.*) result')

In [33]:
df['count'] = df['count'].str.replace(',','')

In [34]:
df['count'] = df['count'].str.replace('About ','')

In [ ]:
df[~df['count'].notnull()]

In [36]:
df['count'] = df['count'].fillna(0)

In [37]:
df.loc[1849,['count']] = 124000

In [ ]:
df[1847:]

In [39]:
df['count'] = pd.to_numeric(df['count'])

In [ ]:
df.dtypes

In [ ]:
df['count'].describe()

In [ ]:
df

In [43]:
df = cnps.merge(df,how='left',left_index=True,right_index=True)

In [ ]:
df.columns

In [45]:
df = df[['Scientific Name','Common Name','CRPR','count']]

In [ ]:
df

In [21]:
import seaborn as sns

In [48]:
df['CRPRgen'] = df['CRPR'].str.extract('(..)')

In [ ]:
sns.boxplot(x='CRPRgen',y='count',data=df[df['count']<1000])


In [ ]:
df

In [ ]:
df['count'][df['count'] < 100000]

In [ ]:
import matplotlib.pyplot as plt


fig, ax = plt.subplots()
df['count'][df['count'] < 100000].hist(ax=ax, bins=50, bottom=0.1)
ax.set_yscale('log')

In [279]:
df.to_csv('RareSppCounts20210307.csv')

In [ ]:
df[df['count']>5000]

In [69]:
import scipy.stats as stats
